In [41]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Set the correct file paths
base_path = r"C:\Users\Wolfrank\Desktop\GiGabyte\CodeWolf\UKDataProject\Data\Combined_Data"
output_path = r"C:\Users\Wolfrank\Desktop\GiGabyte\CodeWolf\UKDataProject\Data\Output"

# Create output directory if it doesn't exist
Path(output_path).mkdir(parents=True, exist_ok=True)

# Read the datasets with selected columns
accidents_df = pd.read_csv(f"{base_path}/combined_accidents.csv",
                          usecols=['Accident_Index', 'Date', 'Time', 'Day_of_Week', 
                                 'Accident_Severity', 'Weather_Conditions', 'Light_Conditions',
                                 'Road_Type', 'Speed_limit', 'Urban_or_Rural_Area',
                                 'Number_of_Vehicles', 'Number_of_Casualties'])

casualties_df = pd.read_csv(f"{base_path}/combined_casualties.csv",
                          usecols=['Accident_Index', 'Age_of_Casualty', 
                                 'Casualty_Severity', 'Casualty_Type',
                                 'Sex_of_Casualty', 'Casualty_Class'])

vehicles_df = pd.read_csv(f"{base_path}/combined_vehicles.csv",
                         usecols=['Accident_Index', 'Vehicle_Type', 
                                'Age_of_Vehicle', 'Age_of_Driver',
                                'Journey_Purpose_of_Driver'])

# Safe time parsing function
def parse_time(time_str):
    try:
        if pd.isna(time_str) or time_str == 'Unknown':
            return np.nan
        return pd.to_datetime(time_str, format='%H:%M').hour
    except:
        return np.nan

# Data preprocessing
accidents_df['Date'] = pd.to_datetime(accidents_df['Date'], format='%d/%m/%Y')
accidents_df['Month'] = accidents_df['Date'].dt.month
accidents_df['Year'] = accidents_df['Date'].dt.year
accidents_df['Hour'] = accidents_df['Time'].apply(parse_time)

# Color palette and style settings
blue_grey_palette = ['#1f77b4', '#7393B3', '#B0C4DE', '#E6E6FA']
plt.style.use('seaborn-darkgrid')
sns.set_palette(blue_grey_palette)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = '#f8f9fa'
plt.rcParams['grid.color'] = '#e9ecef'

# Generate all visualizations
def create_visualizations():
    # 1. Severity by Road Type
    plt.figure(figsize=(15, 8))
    road_severity = pd.crosstab(accidents_df['Road_Type'], accidents_df['Accident_Severity'])
    sns.heatmap(road_severity, cmap='Blues', annot=True, fmt='d')
    plt.title('Accident Severity by Road Type', pad=20, fontsize=14)
    plt.xlabel('Severity', fontsize=12)
    plt.ylabel('Road Type', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{output_path}/road_severity_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 2. Monthly Trends
    plt.figure(figsize=(12, 6))
    monthly_trend = accidents_df.groupby(['Month', 'Accident_Severity']).size().unstack()
    monthly_trend.plot(kind='line', marker='o')
    plt.title('Monthly Accident Trends by Severity', pad=20, fontsize=14)
    plt.xlabel('Month', fontsize=12)
    plt.ylabel('Number of Accidents', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(title='Severity')
    plt.tight_layout()
    plt.savefig(f'{output_path}/monthly_trends.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 3. Time of Day Analysis
    plt.figure(figsize=(15, 8))
    time_severity = pd.crosstab(accidents_df['Hour'], accidents_df['Accident_Severity'])
    sns.heatmap(time_severity, cmap='Blues', annot=True, fmt='d')
    plt.title('Accident Severity by Hour of Day', pad=20, fontsize=14)
    plt.xlabel('Severity', fontsize=12)
    plt.ylabel('Hour of Day', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{output_path}/time_severity.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. Weather Impact
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=accidents_df, x='Weather_Conditions', y='Number_of_Casualties',
                hue='Accident_Severity', palette=blue_grey_palette)
    plt.title('Casualties by Weather Conditions and Severity', pad=20, fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{output_path}/weather_casualties.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 5. Speed Limit Analysis
    plt.figure(figsize=(12, 6))
    speed_severity = pd.crosstab(accidents_df['Speed_limit'], 
                                accidents_df['Accident_Severity'], 
                                normalize='index') * 100
    speed_severity.plot(kind='bar', stacked=True, color=blue_grey_palette)
    plt.title('Accident Severity by Speed Limit (%)', pad=20, fontsize=14)
    plt.xlabel('Speed Limit', fontsize=12)
    plt.ylabel('Percentage', fontsize=12)
    plt.legend(title='Severity')
    plt.tight_layout()
    plt.savefig(f'{output_path}/speed_severity.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 6. Casualty Age Analysis
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=casualties_df, x='Casualty_Type', y='Age_of_Casualty',
                hue='Casualty_Severity', palette=blue_grey_palette)
    plt.title('Age Distribution by Casualty Type and Severity', pad=20, fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{output_path}/casualty_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 7. Vehicle Type Analysis
    plt.figure(figsize=(12, 6))
    vehicle_counts = vehicles_df['Vehicle_Type'].value_counts().head(10)
    sns.barplot(x=vehicle_counts.index, y=vehicle_counts.values, palette=blue_grey_palette)
    plt.title('Top 10 Vehicle Types in Accidents', pad=20, fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{output_path}/vehicle_types.png', dpi=300, bbox_inches='tight')
    plt.close()

    # Additional visualization for age distribution
plt.figure(figsize=(15, 8))
# Create age groups
casualties_df['Age_Group'] = pd.cut(casualties_df['Age_of_Casualty'], 
                                  bins=[0, 16, 25, 35, 45, 55, 65, 75, 100],
                                  labels=['0-16', '17-25', '26-35', '36-45', '46-55', 
                                        '56-65', '66-75', '75+'])

# Create stacked histogram
accident_ages = pd.crosstab(casualties_df['Age_Group'], 
                           casualties_df['Casualty_Severity'])
ax = accident_ages.plot(kind='bar', stacked=True, color=blue_grey_palette)
plt.title('Casualty Distribution by Age Group and Severity', pad=20, fontsize=14)
plt.xlabel('Age Group', fontsize=12)
plt.ylabel('Number of Casualties', fontsize=12)
plt.legend(title='Severity')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f'{output_path}/age_distribution.png', dpi=300, bbox_inches='tight')
plt.close()
    
# Calculate summary statistics
def calculate_statistics():
    return {
        'total_accidents': len(accidents_df),
        'total_casualties': len(casualties_df),
        'fatal_accidents': len(accidents_df[accidents_df['Accident_Severity'] == 1]),
        'serious_accidents': len(accidents_df[accidents_df['Accident_Severity'] == 2]),
        'slight_accidents': len(accidents_df[accidents_df['Accident_Severity'] == 3])
    }
    

# Create visualizations
create_visualizations()
summary_stats = calculate_statistics()



C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_25152\1143794347.py:48: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


<Figure size 1500x800 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

In [52]:


# Generate HTML report
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>UK Traffic Accident Analysis (2015-2018)</title>
    <style>
        body {{ 
            font-family: 'Segoe UI', Arial, sans-serif;
            line-height: 1.6;
            color: #333;
            margin: 0;
            padding-top: 60px;
            background-color: #f8f9fa;
        }}
        
        .nav-bar {{
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            background-color: #000;
            padding: 15px 0;
            z-index: 1000;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        
        .content {{
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
        }}
        
        .nav-links {{
            max-width: 1200px;
            margin: 0 auto;
            padding: 0 20px;
        }}
        
        .nav-link {{
            color: white;
            text-decoration: none;
            padding: 10px 20px;
            margin: 0 5px;
            transition: background-color 0.3s ease;
        }}
        
        .nav-link:hover {{
            background-color: #333;
            border-radius: 4px;
        }}
        
        .section {{
            background: white;
            padding: 30px;
            margin: 30px 0;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        
        h1, h2 {{
            color: #1f77b4;
            margin-bottom: 20px;
        }}
        
        h3 {{
            color: #2c3e50;
            margin: 20px 0 10px 0;
        }}
        
        img {{
            max-width: 100%;
            height: auto;
            margin: 20px 0;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        
        .intro {{
            background: linear-gradient(135deg, #1f77b4, #2c3e50);
            color: white;
            padding: 40px;
            border-radius: 8px;
            margin-bottom: 30px;
        }}
        
        .intro h1 {{
            color: white;
            margin-bottom: 20px;
        }}
        
        .findings {{
            background-color: #f8f9fa;
            padding: 20px;
            margin: 20px 0;
            border-left: 4px solid #1f77b4;
            border-radius: 4px;
        }}
        
        .stats-container {{
            display: flex;
            flex-wrap: wrap;
            gap: 20px;
            margin: 20px 0;
        }}
        
        .stat-card {{
            background: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            flex: 1;
            min-width: 200px;
            text-align: center;
        }}
        
        .stat-value {{
            font-size: 24px;
            font-weight: bold;
            color: #1f77b4;
            margin-bottom: 10px;
        }}
        
        ul, ol {{
            margin: 10px 0;
            padding-left: 20px;
        }}
        
        li {{
            margin-bottom: 8px;
        }}
          .dictionary-section {{
        margin: 30px 0;
    }}

    .table-wrapper {{
        overflow-x: auto;
        margin: 20px 0;
    }}

    .dictionary-table {{
        width: 100%;
        border-collapse: collapse;
        margin: 20px 0;
        background: white;
        box-shadow: 0 1px 3px rgba(0,0,0,0.1);
    }}

    .dictionary-table th,
    .dictionary-table td {{
        padding: 12px 15px;
        text-align: left;
        border-bottom: 1px solid #e0e0e0;
    }}

    .dictionary-table th {{
        background-color: #1f77b4;
        color: white;
        font-weight: 500;
    }}

    .dictionary-table tr:nth-child(even) {{
        background-color: #f8f9fa;
    }}

    .dictionary-table tr:hover {{
        background-color: #f5f5f5;
    }}
    
         .grid-container {{
            display: grid;
            grid-template-columns: auto auto;
            grid-gap: 10px;
        }}

        .grid-item {{
            text-align: center;
        }}
 
        
    </style>
</head>
<body>
    <nav class="nav-bar">
        <div class="nav-links">
            <a href="#introduction" class="nav-link">Introduction</a>
            <a href="#methodology" class="nav-link">Methodology</a>
            <a href="#findings" class="nav-link">Findings</a>
            <a href="#conclusion" class="nav-link">Conclusion</a>
        </div>
    </nav>

    <div class="content">
    <div id="introduction" class="intro">
    <h1>UK Traffic Accident Analysis (2015-2018)</h1>
    <p>Road safety remains one of the most critical public health challenges in the United Kingdom, affecting thousands of lives annually through accidents, injuries, and fatalities. This comprehensive analysis examines traffic accident data from 2015 to 2018, seeking to understand the complex interplay of factors that contribute to road incidents and their severity. By analyzing over {summary_stats['total_accidents']:,} accidents, this study provides crucial insights into patterns, risk factors, and potential intervention points for improving road safety.</p>
    
    <p>The significance of this analysis extends beyond mere statistics, touching upon fundamental aspects of public safety, urban planning, and social responsibility. Through detailed examination of temporal patterns, environmental conditions, and demographic factors, we uncover the multifaceted nature of traffic accidents. The findings reveal how various elements - from weather conditions and road types to driver demographics and vehicle characteristics - interact to influence both the likelihood and severity of accidents. This understanding is crucial for developing targeted interventions and evidence-based policies to enhance road safety.</p>
    
    <p>This study employs advanced statistical analysis and visualization techniques to decode complex patterns within the data, focusing particularly on identifying high-risk scenarios and vulnerable populations. By examining the relationship between factors such as time of day, weather conditions, road types, and accident severity, we aim to provide actionable insights for policymakers, urban planners, and road safety professionals. The analysis pays special attention to age-related patterns, environmental impacts, and vehicular factors, offering a comprehensive view of road safety challenges in the UK.</p>
            
    <h3>Research Objectives:</h3>
    <ul>
        <li>Identify temporal and seasonal patterns in accident occurrence</li>
        <li>Analyze the impact of environmental and road conditions</li>
        <li>Evaluate demographic patterns in casualties</li>
        <li>Assess vehicle-specific risk factors</li>
        <li>Develop evidence-based safety recommendations</li>
    </ul>
</div>

        <div id="methodology" class="section">
            <h2>Methodology and Data Overview</h2>
            <div class="stats-container">
                <div class="stat-card">
                    <div class="stat-value">{summary_stats['total_accidents']:,}</div>
                    <div>Total Accidents</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{summary_stats['fatal_accidents']:,}</div>
                    <div>Fatal Accidents</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{summary_stats['serious_accidents']:,}</div>
                    <div>Serious Accidents</div>
                </div>
                <div class="stat-card">
                    <div class="stat-value">{summary_stats['slight_accidents']:,}</div>
                    <div>Slight Accidents</div>
                </div>
            </div>
            <p>This analysis utilizes comprehensive accident data from 2015 to 2018, incorporating multiple data sources to provide a holistic view of road safety incidents.</p>
        </div>
        <div id="data-dictionary" class="section">
    <h2>Data Dictionary</h2>
    
    <div class="dictionary-section">
        <h3>Accidents Dataset</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Field Name</th>
                        <th>Description</th>
                        <th>Data Type</th>
                        <th>Values/Format</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td>Accident_Index</td>
                        <td>Unique identifier for each accident</td>
                        <td>String</td>
                        <td>Unique value linking to vehicle and casualty data</td>
                    </tr>
                    <tr>
                        <td>Location_Easting_OSGR</td>
                        <td>Easting location in OSGR format</td>
                        <td>Numeric</td>
                        <td>Grid Reference (-1 for missing data)</td>
                    </tr>
                    <tr>
                        <td>Location_Northing_OSGR</td>
                        <td>Northing location in OSGR format</td>
                        <td>Numeric</td>
                        <td>Grid Reference (-1 for missing data)</td>
                    </tr>
                    <tr>
                        <td>Longitude</td>
                        <td>Longitude in WGS84 format</td>
                        <td>Decimal</td>
                        <td>WGS 1984 coordinate system</td>
                    </tr>
                    <tr>
                        <td>Latitude</td>
                        <td>Latitude in WGS84 format</td>
                        <td>Decimal</td>
                        <td>WGS 1984 coordinate system</td>
                    </tr>
                    <tr>
                        <td>Accident_Severity</td>
                        <td>Severity of the accident</td>
                        <td>Integer</td>
                        <td>1: Fatal, 2: Serious, 3: Slight</td>
                    </tr>
                    <tr>
                        <td>Number_of_Vehicles</td>
                        <td>Number of vehicles involved</td>
                        <td>Integer</td>
                        <td>Count of vehicles</td>
                    </tr>
                    <tr>
                        <td>Number_of_Casualties</td>
                        <td>Number of casualties</td>
                        <td>Integer</td>
                        <td>Count of casualties</td>
                    </tr>
                    <tr>
                        <td>Date</td>
                        <td>Date of accident</td>
                        <td>Date</td>
                        <td>DD/MM/YYYY format</td>
                    </tr>
                    <tr>
                        <td>Time</td>
                        <td>Time of accident</td>
                        <td>Time</td>
                        <td>HH:MM 24-hour format</td>
                    </tr>
                </tbody>
            </table>
        </div>

        <h3>Casualties Dataset</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Field Name</th>
                        <th>Description</th>
                        <th>Data Type</th>
                        <th>Values/Format</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td>Accident_Index</td>
                        <td>Reference to accident record</td>
                        <td>String</td>
                        <td>Links to accident data</td>
                    </tr>
                    <tr>
                        <td>Vehicle_Reference</td>
                        <td>Reference to vehicle involved</td>
                        <td>Integer</td>
                        <td>Links to vehicle data</td>
                    </tr>
                    <tr>
                        <td>Casualty_Class</td>
                        <td>Type of casualty</td>
                        <td>Integer</td>
                        <td>1: Driver/Rider, 2: Passenger, 3: Pedestrian</td>
                    </tr>
                    <tr>
                        <td>Sex_of_Casualty</td>
                        <td>Gender of casualty</td>
                        <td>Integer</td>
                        <td>1: Male, 2: Female, -1: Unknown</td>
                    </tr>
                    <tr>
                        <td>Age_of_Casualty</td>
                        <td>Age of casualty</td>
                        <td>Integer</td>
                        <td>Age in years (-1 for unknown)</td>
                    </tr>
                </tbody>
            </table>
        </div>

        <h3>Vehicles Dataset</h3>
        <div class="table-wrapper">
            <table class="dictionary-table">
                <thead>
                    <tr>
                        <th>Field Name</th>
                        <th>Description</th>
                        <th>Data Type</th>
                        <th>Values/Format</th>
                    </tr>
                </thead>
                <tbody>
                    <tr>
                        <td>Accident_Index</td>
                        <td>Reference to accident record</td>
                        <td>String</td>
                        <td>Links to accident data</td>
                    </tr>
                    <tr>
                        <td>Vehicle_Type</td>
                        <td>Type of vehicle</td>
                        <td>Integer</td>
                        <td>Various codes for vehicle types</td>
                    </tr>
                    <tr>
                        <td>Age_of_Driver</td>
                        <td>Age of driver</td>
                        <td>Integer</td>
                        <td>Age in years (-1 for unknown)</td>
                    </tr>
                    <tr>
                        <td>Age_of_Vehicle</td>
                        <td>Age of vehicle</td>
                        <td>Integer</td>
                        <td>Age in years (-1 for unknown)</td>
                    </tr>
                </tbody>
            </table>
        </div>
    </div>
</div>

        <div id="findings" class="section">
            <h2>Detailed Analysis and Findings</h2>
            
            <h3>1. Temporal Analysis</h3>
            <img src="time_severity.png" alt="Time of Day Analysis">
            <div class="findings">
                <h4>Key Observations:</h4>
                <ul>
                    <li>Peak accident times coincide with rush hours (7-9 AM and 4-6 PM)</li>
                    <li>Highest severity rates occur during nighttime hours (11 PM - 4 AM)</li>
                    <li>Early morning hours (2-5 AM) show lowest frequency but higher severity</li>
                    <li>Weekend patterns differ significantly from weekday patterns</li>
                </ul>
            </div>

            <h3>2. Weather Impact Analysis</h3>
            <img src="weather_casualties.png" alt="Weather Analysis">
            <div class="findings">
                <h4>Key Findings:</h4>
                <ul>
                    <li>Adverse weather conditions significantly impact accident severity</li>
                    <li>Rain is associated with increased accident frequency but lower average severity</li>
                    <li>Snow and ice show lower frequency but higher severity rates</li>
                    <li>Clear weather accounts for majority of accidents due to higher traffic volume</li>
                </ul>
            </div>

            <h3>3. Road and Speed Analysis</h3>
            <img src="road_severity_heatmap.png" alt="Road Type Analysis">
            <img src="speed_severity.png" alt="Speed Analysis">
            <div class="findings">
                <h4>Critical Insights:</h4>
                <ul>
                    <li>Single carriageways account for highest number of accidents</li>
                    <li>Higher speed limits correlate strongly with increased severity</li>
                    <li>Urban roads show higher frequency but lower severity patterns</li>
                    <li>Motorways show relatively low accident rates despite high speeds</li>
                </ul>
            </div>

            <h3>4. Casualty Analysis</h3>
            <img src="casualty_analysis.png" alt="Casualty Analysis">
            <div class="findings">
                <h4>Demographic Patterns:</h4>
                <ul>
                    <li>Young adults (18-25) show higher representation in accidents</li>
                    <li>Elderly casualties (65+) show higher severity rates</li>
                    <li>Pedestrian casualties show distinct age distribution patterns</li>
                    <li>Cyclist casualties concentrate in urban areas and peak commuting times</li>
                </ul>
            </div>

            <h3>5. Vehicle Type Analysis</h3>
            <img src="vehicle_types.png" alt="Vehicle Analysis">
            <div class="findings">
                <h4>Vehicle-Specific Patterns:</h4>
                <ul>
                    <li>Cars represent the majority of vehicles involved</li>
                    <li>Motorcycles show disproportionately high severity rates</li>
                    <li>Heavy goods vehicles involved in fewer but more severe accidents</li>
                    <li>Public transport shows lower involvement rates</li>
                </ul>
            </div>
        </div>
        <h3>6. Age Distribution Analysis</h3>
<img src="age_distribution.png" alt="Age Distribution Analysis">
<div class="findings">
    <h4>Age-Related Patterns:</h4>
    <ul>
        <li>Young adults (17-25) show highest accident involvement</li>
        <li>Severity rates increase with age groups</li>
        <li>Children under 16 show distinct accident patterns</li>
        <li>Middle-aged groups show more moderate risk levels</li>
    </ul>
    
</div>
<!------>
  <div><h1> Accident Correlations </h1></div>

  <div class="grid-container">
        <div class="grid-item">
            <a href="/Data/Output/hourly_distribution.png" target="_blank">
                <img src="/Data/Output/hourly_distribution.png" alt="Hourly Distribution">
            </a>
            <p>Hourly Distribution</p>
        </div>
        <div class="grid-item">
            <a href="/Data/Output/monthly_severity_trends.png" target="_blank">
                <img src="/Data/Output/monthly_severity_trends.png" alt="Monthly Severity Trends">
            </a>
            <p>Monthly Severity Trends</p>
        </div>
        <div class="grid-item">
            <a href="/Data/Output/severity_heatmap.png" target="_blank">
                <img src="/Data/Output/severity_heatmap.png" alt="Severity Heatmap">
            </a>
            <p>Severity Heatmap</p>
        </div>
        <div class="grid-item">
            <a href="/Data/Output/time_of_day.png" target="_blank">
                <img src="/Data/Output/time_of_day.png" alt="Time of Day">
            </a>
            <p>Time of Day</p>
        </div>
        <div class="grid-item">
            <a href="/Data/Output/time_severity.png" target="_blank">
                <img src="/Data/Output/time_severity.png" alt="Time Severity">
            </a>
            <p>Time Severity</p>
        </div>
        <div class="grid-item">
            <a href="/Data/Output/vehicle_types.png" target="_blank">
                <img src="/Data/Output/vehicle_types.png" alt="Vehicle Types">
            </a>
            <p>Vehicle Types</p>
        </div>
    </div>

<!----->


        <div id="conclusion" class="section">
            <h2>Conclusions and Recommendations</h2>
            
            <div class="findings">
                <h3>Primary Conclusions:</h3>
                <ol>
                    <li><strong>Temporal Factors</strong>
                        <ul>
                            <li>Rush hour periods require enhanced traffic management</li>
                            <li>Night driving presents significantly higher risks</li>
                            <li>Seasonal patterns affect accident likelihood</li>
                        </ul>
                    </li>
                    
                    <li><strong>Environmental Impact</strong>
                        <ul>
                            <li>Weather conditions are crucial risk factors</li>
                            <li>Road type and condition significantly influence outcomes</li>
                            <li>Light conditions play a vital role in accident severity</li>
                        </ul>
                    </li>
                    
                    <li><strong>Human Factors</strong>
                        <ul>
                            <li>Age and experience significantly impact risk levels</li>
                            <li>Different road user types show distinct risk patterns</li>
                            <li>Time of travel affects risk exposure</li>
                        </ul>
                    </li>
                </ol>
            </div>

            <div class="findings">
                <h3>Recommendations:</h3>
                <ul>
                    <li>Implement targeted safety measures during high-risk periods</li>
                    <li>Enhance infrastructure in identified high-risk areas</li>
                    <li>Develop specific safety campaigns for vulnerable groups</li>
                    <li>Improve weather-related warning systems</li>
                    <li>Review speed limits in areas with high severity rates</li>
                </ul>
            </div>

            <div class="findings">
                <h3>Future Research Directions:</h3>
                <ul>
                    <li>Impact of emerging vehicle technologies on safety</li>
                    <li>Detailed analysis of behavioral factors</li>
                    <li>Effectiveness of various safety interventions</li>
                    <li>Integration with real-time traffic and weather data</li>
                </ul>
            </div>
        </div>
    </div>
</body>
</html>
"""

# Save the report
with open(f'{output_path}/report.html', 'w') as f:
    f.write(html_content)

print(f"\nAnalysis complete! Output files have been saved to: {output_path}")
print("Open 'report.html' in a web browser to view the complete analysis.")


Analysis complete! Output files have been saved to: C:\Users\Wolfrank\Desktop\GiGabyte\CodeWolf\UKDataProject\Data\Output
Open 'report.html' in a web browser to view the complete analysis.
